In [25]:
#  Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import pprint

#  Importing API key
from api_keys import bea_api_key

base_url = "https://apps.bea.gov/api/data/"

In [26]:
# BEA CLient for Industry Analysis

class BEAClientIndustry:

  def __init__(self, api_key):
    self.api_key = api_key
    self.base_url = base_url
    self.common_params = {
        'UserID': self.api_key,
        'method': 'GetData',
        'ResultFormat': 'JSON'
    }
    
  def make_request(self, params):
    try:
      full_params = {**self.common_params, **params}
      response = requests.get(self.base_url, params=full_params)
      response.raise_for_status()
      return response.json()
    except requests.exceptions.RequestException as e:
      print(f"API request failed: {e}")
      return None

  def get_gdp_industry_data(self, years=['2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']):

    params = {
        'datasetname': 'GDPbyIndustry',
        'Year': ','.join(years),
        'Industry': 'ALL',
        'TableID': '5',  # Value Added by Industry
        'Frequency': 'Q'  
        }
    return self.make_request(params)
  
# Initializing Client
industry_analyzer = BEAClientIndustry(bea_api_key)
print("BEA Client initialized")

BEA Client initialized


In [27]:
def process_gdpIndustry_data(raw_data):
    if not raw_data or 'BEAAPI' not in raw_data:
        print('No Data Received. Invalid Input')
        return None 

    try:
        
        results_list = raw_data['BEAAPI']['Results']
        
        if not results_list or not isinstance(results_list, list):
            print("Results is not a list or is empty")
            return None
            
        results = results_list[0]['Data'] 
        
        processed_data = []
        for item in results:
            try:
                value = float(item['DataValue'].replace(',', ''))
            except (ValueError, AttributeError):
                continue
                
            processed_data.append({
                'Industry % of GDP': value,
                'Industry Description': item['IndustrYDescription'], 
                'Industry': item['Industry'],
                'NoteRef': item['NoteRef'],
                'Quarter': item['Quarter'],
                'TableID': item['TableID'],
                'Year': item['Year']
            })

        gdp_by_industry_df = pd.DataFrame(processed_data)
        return gdp_by_industry_df

    except Exception as e:
        print(f"Error processing data: {e}")
        return None

In [28]:
gdp_response = industry_analyzer.get_gdp_industry_data()
industry_percent_df = process_gdpIndustry_data(gdp_response)
industry_percent_df.reset_index(drop=True, inplace=True)
industry_percent_df

,Industry % of GDP,Industry Description,Industry,NoteRef,Quarter,TableID,Year
0,0.9,"Agriculture, forestry, fishing, and hunting",11,5,I,5,2018
1,0.7,Farms,111CA,5,I,5,2018
2,0.2,"Forestry, fishing, and related activities",113FF,5,I,5,2018
3,1.5,Mining,21,5,I,5,2018
4,0.9,Oil and gas extraction,211,5,I,5,2018
...,...,...,...,...,...,...,...
2866,2.8,Other real estate,ORE,5,I,5,2025
2867,16.4,Private goods-producing industries<sup>1</sup>,PGOOD,5;5.1.Q,I,5,2025
2868,13.3,Professional and business services,PROF,5,I,5,2025
2869,72.3,Private services-producing industries<sup>2</sup>,PSERV,5;5.2.Q,I,5,2025


In [29]:
# returning a list of all the industries in the df
industry_list = industry_percent_df['Industry Description'].unique()
print(industry_list)

['Agriculture, forestry, fishing, and hunting' 'Farms'
 'Forestry, fishing, and related activities' 'Mining'
 'Oil and gas extraction' 'Mining, except oil and gas'
 'Support activities for mining' 'Utilities' 'Construction'
 'Food and beverage and tobacco products'
 'Textile mills and textile product mills'
 'Apparel and leather and allied products' 'Manufacturing'
 'Nondurable goods' 'Wood products' 'Paper products'
 'Printing and related support activities' 'Petroleum and coal products'
 'Chemical products' 'Plastics and rubber products'
 'Nonmetallic mineral products' 'Primary metals'
 'Fabricated metal products' 'Machinery'
 'Computer and electronic products'
 'Electrical equipment, appliances, and components'
 'Motor vehicles, bodies and trailers, and parts'
 'Other transportation equipment' 'Furniture and related products'
 'Miscellaneous manufacturing' 'Durable goods' 'Wholesale trade'
 'Motor vehicle and parts dealers' 'Food and beverage stores'
 'Retail trade' 'General merchan

In [30]:
print(type(industry_list))

<class 'numpy.ndarray'>


In [31]:
total_elements = industry_list.size
print(total_elements)

97


In [32]:
industry_count = industry_percent_df['Industry Description'].nunique()
print(industry_count)

97


In [33]:
# writing to an excel file for future reference
# gdp_df.to_excel('Industry_Percent_of_GDP.xlsx', header=True, index=False)

In [34]:
# limiting the data to Q1 2025 to find main sectors
data_2025 = industry_percent_df[(industry_percent_df['Year'] == '2025')].sort_values(by='Industry % of GDP', ascending=False)
data_2025.reset_index(drop=True, inplace=True)
data_2025

,Industry % of GDP,Industry Description,Industry,NoteRef,Quarter,TableID,Year
0,100.0,Gross domestic product,GDP,5,I,5,2025
1,88.6,Private industries,PVT,5,I,5,2025
2,72.3,Private services-producing industries<sup>2</sup>,PSERV,5;5.2.Q,I,5,2025
3,21.3,"Finance, insurance, real estate, rental, and l...",FIRE,5,I,5,2025
4,16.4,Private goods-producing industries<sup>1</sup>,PGOOD,5;5.1.Q,I,5,2025
...,...,...,...,...,...,...,...
94,0.1,Textile mills and textile product mills,313TT,5,I,5,2025
95,0.1,Water transportation,483,5,I,5,2025
96,0.1,"Funds, trusts, and other financial vehicles",525,5,I,5,2025
97,0.1,Printing and related support activities,323,5,I,5,2025


In [35]:
# summing the entire column to get a baseline of combinations
total_percent = data_2025['Industry % of GDP'].sum()
total_percent

545.2

In [36]:
large_sectors = data_2025[(data_2025['Industry % of GDP'] >= 50)].count()['Industry Description']
large_sectors

3

In [37]:
def clean_data(df):
  # major sectors listed BEA industry reports
  major_sectors = ['Finance and insurance',
                   'Agriculture, forestry, fishing, and hunting',
                   'Wholesale trade',
                   'Retail trade',
                   'Mining',
                   'Other services, except government',
                   'Management of companies and enterprises',
                   'Utilities',
                   'Arts, entertainment, and recreation',
                   'Accommodation and food services',
                   'Transportation and warehousing',
                   'Educational services',
                   'Construction',
                   'Nondurable goods manufacturing',
                   'Administrative and waste management',
                   'Federal government',
                   'Durable goods manufacturing',
                   'Professional, scienctific. and technical services',
                   'Health care and social assistance',
                   'State and local government',
                   'Information',
                   'Real estate and rental leasing']

  df_filtered = df[df['Industry Description'].isin(major_sectors)].copy()
  return df_filtered

major_sectors_df = clean_data(industry_percent_df)
major_sectors_df

,Industry % of GDP,Industry Description,Industry,NoteRef,Quarter,TableID,Year
0,0.9,"Agriculture, forestry, fishing, and hunting",11,5,I,5,2018
3,1.5,Mining,21,5,I,5,2018
7,1.6,Utilities,22,5,I,5,2018
8,4.3,Construction,23,5,I,5,2018
31,5.9,Wholesale trade,42,5,I,5,2018
...,...,...,...,...,...,...,...
2840,1.1,Educational services,61,5,I,5,2025
2841,7.7,Health care and social assistance,62,5,I,5,2025
2847,1.2,"Arts, entertainment, and recreation",71,5,I,5,2025
2850,3.3,Accommodation and food services,72,5,I,5,2025
